In [1]:
from vpl_extraction.vpl_star import VPLStar
from examples.dyck1 import Dyck1

In [2]:
dyck1_grammar = Dyck1()

vpl_star = VPLStar(dyck1_grammar)

In [3]:
result = vpl_star.learn()

In [4]:
print(str(result.tree_automaton))

TreeAutomata( 
 -- states=[") ['ε []']", 'ε []'] 
 -- final_states=['ε []'] 
 -- transitions=['Key(ε, [])', "Key(), ['ε []'])"] 
)


In [5]:
from models.vpg import vpg_from_tree_automata

vpg = vpg_from_tree_automata(result.tree_automaton, dyck1_grammar.alphabet)

print(vpg.print_grammar())

Grammar VPG
Variables: q1, q0
Start Symbols: q1
Rules:
q1 -> ε    # epsilon


Variable Map:
) ['ε []'] -> q0
ε [] -> q1



In [6]:
i=0
for k, v in result.tree_automaton.transitions.items():
    i+=1
    print(f"{i}. {k.symbol} {[(vpg.variable_map[str(c)]) for c in k.child_states]} ---> Val {vpg.variable_map[str(v)]}")

1. ε [] ---> Val q1
2. ) ['q1'] ---> Val q0


In [7]:
random_word_count = 10000
counterexample = False
for _ in range(random_word_count):
    word = dyck1_grammar.get_random_word()
    
    if result.is_accepted(word) != dyck1_grammar.is_accepted(word):
        print(f"VPLStar does not match Dyck1 for the word: {word}")
        counterexample = True
        break

if not counterexample:
    print("VPLStar matches Dyck1 for all tested words.")

VPLStar does not match Dyck1 for the word: ()()


In [8]:
alphabet = Dyck1().alphabet

In [9]:
from experiments.transformer_checker_wrapper import TransformerWrapper

transformer = TransformerWrapper(
    metadata_path='./experiments/models/metadata/transformer_model.json',
    alphabet=alphabet
)

/Users/juanpe/miniconda3/envs/TransformerExtraction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: juancommits. Use `wandb login --relogin` to force relogin


In [10]:
from vpl_extraction.vpl_star_random_comparator import VPLRandomComparator
comparator = VPLRandomComparator(
    alphabet=alphabet,
    random_trees=500,
    max_depth=8
)

In [11]:
vpl_star = VPLStar(transformer, comparator)

In [12]:
res = vpl_star.learn()

Tokenizing strings: 100%|██████████| 1/1 [00:00<00:00, 4826.59it/s]


In [13]:
print(res.tree_automaton)

TreeAutomata( 
 -- states=['ε []', ") ['ε []']"] 
 -- final_states=['ε []'] 
 -- transitions=['Key((, [\'ε []\', ") [\'ε []\']"])', 'Key(ε, [])', "Key(), ['ε []'])"] 
)


In [14]:
i=0
for k, v in res.tree_automaton.transitions.items():
    i+=1
    print(f"{i}. {k.symbol} {[(vpg.variable_map[str(c)]) for c in k.child_states]} ---> Val {vpg.variable_map[str(v)]}")

1. ( ['q1', 'q0'] ---> Val q1
2. ε [] ---> Val q1
3. ) ['q1'] ---> Val q0


In [15]:
transformer.is_accepted("(())")

Tokenizing strings: 100%|██████████| 1/1 [00:00<00:00, 4481.09it/s]


True

In [16]:
vpg = vpg_from_tree_automata(res.tree_automaton, alphabet)

print(vpg.print_grammar())

Grammar VPG
Variables: q1, q0
Start Symbols: q0
Rules:
q0 -> ε    # epsilon
q0 -> ( q0 ) q0    # push-pop


Variable Map:
ε [] -> q0
) ['ε []'] -> q1

